In [1]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class Word(Base):
    __tablename__ = "word_dict"
    id: Mapped[int] = mapped_column(primary_key=True)
    word: Mapped[str] 
    tonic_accent: Mapped[str]
    comment: Mapped[Optional[str]]
    index_ref_entries: Mapped[List["IndexEntry"]] = relationship(
        back_populates="word", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"Word(id={self.id!r}, word={self.word!r}, comment={self.comment!r})"

class IndexEntry(Base):
    __tablename__ = "word_index"
    id : Mapped[int] = mapped_column(primary_key=True)
    lesson : Mapped[int]
    line : Mapped[int]
    position : Mapped[Optional[int]]
    word_id : Mapped[int] = mapped_column(ForeignKey("word_dict.id"))
    word : Mapped["Word"] = relationship(back_populates="index_ref_entries")
    def __repr__(self) -> str:
        return f"IndexEntry(id={self.id!r}, word={self.word!r}, word_id={self.word_id}, lesson={self.lesson!r}, line={self.line!r}), position={self.position!r}"
        
class Sentence(Base):
    __tablename__ = "sentences"
    id: Mapped[int] = mapped_column(primary_key=True)
    sentence: Mapped[str] 
    comment: Mapped[Optional[str]]
    lesson : Mapped[int]
    line : Mapped[int]
    def __repr__(self) -> str:
        return f"Sentence(id={self.id!r}, lesson={lesson!r}, numero={numero!r}, sentence ={self.sentence!r}, comment={self.comment!r})"

In [2]:
from pathlib import Path

db_directory = "./db"
word_dict_filename = "test_tonic_accent_word_dict.db"

db_name = Path(db_directory, word_dict_filename)
db_url  = f"sqlite:///{db_name}"
db_url

'sqlite:///db/test_tonic_accent_word_dict.db'

In [3]:
from sqlalchemy import create_engine
engine = create_engine(db_url, echo=True)

In [4]:
Base.metadata.create_all(engine)

2024-08-20 09:23:53,305 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 09:23:53,306 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("word_dict")
2024-08-20 09:23:53,306 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 09:23:53,307 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("word_dict")
2024-08-20 09:23:53,308 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 09:23:53,308 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("word_index")
2024-08-20 09:23:53,309 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 09:23:53,310 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("word_index")
2024-08-20 09:23:53,310 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 09:23:53,311 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sentences")
2024-08-20 09:23:53,311 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 09:23:53,312 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sentences")
2024-08-20 09:23:53,312 INFO sqlalchemy.engine.Engin

In [5]:
from time import time
ti = time()
from tonic_accent import global_word_dict as word_dict
tf = time()
print(f"time to fetch word dictionary from html files : {tf-ti}")


Analyzing lesson 1 with length 521
Analyzing lesson 2 with length 631
Analyzing lesson 3 with length 706
Analyzing lesson 4 with length 670
Analyzing lesson 5 with length 807
Analyzing lesson 6 with length 785
Analyzing lesson 7 with length 751
Analyzing lesson 8 with length 1117
Analyzing lesson 9 with length 1059
Analyzing lesson 10 with length 1064
word_dict.index_ref : [(9, 5), (10, 2), (10, 3)]
Analyzing lesson 11 with length 1216
Analyzing lesson 12 with length 1104
Analyzing lesson 13 with length 1187
word_dict.index_ref : [(9, 5), (10, 2), (10, 3), (13, 6), (13, 14)]
Analyzing lesson 14 with length 954
word_dict.index_ref : [(9, 5), (10, 2), (10, 3), (13, 6), (13, 14), (14, 5)]
Analyzing lesson 15 with length 1058
Analyzing lesson 16 with length 1392
Analyzing lesson 17 with length 1143
Analyzing lesson 18 with length 1307
Analyzing lesson 19 with length 1206
Analyzing lesson 20 with length 1273
Analyzing lesson 21 with length 1297
Analyzing lesson 22 with length 1353
Analyzing

In [6]:
keys_number = 0
for key in word_dict:
    keys_number += 1
print(f"Number of keys : {keys_number}")

Number of keys : 3555


In [7]:
word_dict["casa"]

Word_properties(tonic_accent=[('ca', True), ('sa', False)], index_ref=[(9, 5), (10, 2), (10, 3), (13, 6), (13, 14), (14, 5), (26, 3), (26, 15), (28, 7), (28, 11), (35, 9), (78, 2), (78, 20), (80, 12), (84, 8), (88, 4), (88, 7), (90, 1)])

In [8]:
word_dict["saldríamos"]

Word_properties(tonic_accent=[('sal', False), ('drí', True), ('amos', False)], index_ref=[(80, 3)])

In [9]:
word_dict["niños"]

Word_properties(tonic_accent=[('ni', True), ('ños', False)], index_ref=[(24, 10), (30, 6), (30, 15), (62, 21), (79, 12), (80, 9), (80, 10), (80, 19)])

In [10]:
from sqlalchemy.orm import Session

In [11]:
engine.echo=True

In [12]:
engine.echo

True

In [13]:
from sqlalchemy.orm import Session
import pickle
with Session(engine) as session:
    casa = Word(
        word="casa", 
        tonic_accent=pickle.dumps(word_dict["casa"].index_ref),
        index_ref_entries=[
            IndexEntry(lesson=9, line=5),
            IndexEntry(lesson=10, line=2),
            IndexEntry(lesson=10, line=3),
            IndexEntry(lesson=13, line=6),
            IndexEntry(lesson=13, line=14),
        ],
    )
    saldriamos = Word(
        word="saldríamos", 
        tonic_accent=pickle.dumps(word_dict["saldríamos"].index_ref),
        index_ref_entries=[
            IndexEntry(lesson=80, line=3),
        ]
    )
    ninos = Word(
        word="niños", 
        tonic_accent=pickle.dumps(word_dict["niños"].index_ref),
        index_ref_entries=[
            IndexEntry(lesson=24, line=10),
            IndexEntry(lesson=30, line=6),
            IndexEntry(lesson=30, line=15),
            IndexEntry(lesson=62, line=21),
            IndexEntry(lesson=79, line=12),
        ],
    )
    session.add_all([casa, saldriamos, ninos])
    session.commit()
    
    


2024-08-20 09:19:05,571 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 09:19:05,574 INFO sqlalchemy.engine.Engine INSERT INTO word_dict (word, tonic_accent, comment) VALUES (?, ?, ?) RETURNING id
2024-08-20 09:19:05,575 INFO sqlalchemy.engine.Engine [generated in 0.00017s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('casa', b'\x80\x04\x95q\x00\x00\x00\x00\x00\x00\x00]\x94(K\tK\x05\x86\x94K\nK\x02\x86\x94K\nK\x03\x86\x94K\rK\x06\x86\x94K\rK\x0e\x86\x94K\x0eK\x05\x86\x94K\x ... (40 characters truncated) ... x07\x86\x94K\x1cK\x0b\x86\x94K#K\t\x86\x94KNK\x02\x86\x94KNK\x14\x86\x94KPK\x0c\x86\x94KTK\x08\x86\x94KXK\x04\x86\x94KXK\x07\x86\x94KZK\x01\x86\x94e.', None)
2024-08-20 09:19:05,577 INFO sqlalchemy.engine.Engine INSERT INTO word_dict (word, tonic_accent, comment) VALUES (?, ?, ?) RETURNING id
2024-08-20 09:19:05,577 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('saldríamos', b'\x80\x04\x95\n\x00\x00\x00\x00\x00\x00\x00]\x94K

In [14]:
from sqlalchemy import select
stmt = select(Word).where(Word.word == "niños")

for word in session.scalars(stmt):
    print(word)

2024-08-20 09:19:10,335 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 09:19:10,338 INFO sqlalchemy.engine.Engine SELECT word_dict.id, word_dict.word, word_dict.tonic_accent, word_dict.comment 
FROM word_dict 
WHERE word_dict.word = ?
2024-08-20 09:19:10,339 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ('niños',)
Word(id=3, word='niños', comment=None)


In [15]:
from sqlalchemy.orm import Session
import pickle
with Session(engine) as session:

    stmt = select(IndexEntry).where(IndexEntry.lesson == 62)

    for entry in session.scalars(stmt):
        print(entry)

2024-08-20 09:19:11,245 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 09:19:11,247 INFO sqlalchemy.engine.Engine SELECT word_index.id, word_index.lesson, word_index.line, word_index.position, word_index.word_id 
FROM word_index 
WHERE word_index.lesson = ?
2024-08-20 09:19:11,248 INFO sqlalchemy.engine.Engine [generated in 0.00095s] (62,)
2024-08-20 09:19:11,252 INFO sqlalchemy.engine.Engine SELECT word_dict.id AS word_dict_id, word_dict.word AS word_dict_word, word_dict.tonic_accent AS word_dict_tonic_accent, word_dict.comment AS word_dict_comment 
FROM word_dict 
WHERE word_dict.id = ?
2024-08-20 09:19:11,253 INFO sqlalchemy.engine.Engine [generated in 0.00099s] (3,)
IndexEntry(id=10, word=Word(id=3, word='niños', comment=None), word_id=3, lesson=62, line=21), position=None
2024-08-20 09:19:11,254 INFO sqlalchemy.engine.Engine ROLLBACK


In [16]:
stmt = select(IndexEntry).where(IndexEntry.word_id == 3)

for entry in session.scalars(stmt):
    print(entry)

2024-08-20 09:19:12,476 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 09:19:12,478 INFO sqlalchemy.engine.Engine SELECT word_index.id, word_index.lesson, word_index.line, word_index.position, word_index.word_id 
FROM word_index 
WHERE word_index.word_id = ?
2024-08-20 09:19:12,479 INFO sqlalchemy.engine.Engine [generated in 0.00080s] (3,)
2024-08-20 09:19:12,482 INFO sqlalchemy.engine.Engine SELECT word_dict.id AS word_dict_id, word_dict.word AS word_dict_word, word_dict.tonic_accent AS word_dict_tonic_accent, word_dict.comment AS word_dict_comment 
FROM word_dict 
WHERE word_dict.id = ?
2024-08-20 09:19:12,483 INFO sqlalchemy.engine.Engine [cached since 1.231s ago] (3,)
IndexEntry(id=7, word=Word(id=3, word='niños', comment=None), word_id=3, lesson=24, line=10), position=None
IndexEntry(id=8, word=Word(id=3, word='niños', comment=None), word_id=3, lesson=30, line=6), position=None
IndexEntry(id=9, word=Word(id=3, word='niños', comment=None), word_id=3, lesson=30, line=15),

In [7]:
from sqlalchemy.orm import Session
import pickle
engine.echo=False
with Session(engine) as session:
    for word in word_dict:
        entries = []
        for entry in word_dict[word].index_ref:
            lesson, line = entry
            entries.append(IndexEntry(lesson=lesson, line=line))
        word_obj = Word(
            word=word, 
            tonic_accent=pickle.dumps(word_dict[word].tonic_accent),
            index_ref_entries=entries
        )
        session.add_all([word_obj])
        session.commit()
    


In [8]:
from sqlalchemy import select
stmt = select(Word).where(Word.word == "niños")

for word in session.scalars(stmt):
    print(word)
    for entry in word.index_ref_entries:
        print(f"lesson : {entry.lesson}, line : {entry.line}")

Word(id=639, word='niños', comment=None)
lesson : 24, line : 10
lesson : 30, line : 6
lesson : 30, line : 15
lesson : 62, line : 21
lesson : 79, line : 12
lesson : 80, line : 9
lesson : 80, line : 10
lesson : 80, line : 19


In [9]:
stmt = select(IndexEntry).where(IndexEntry.word_id == 639)

for entry in session.scalars(stmt):
    print(entry)

IndexEntry(id=13226, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=24, line=10), position=None
IndexEntry(id=13227, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=30, line=6), position=None
IndexEntry(id=13228, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=30, line=15), position=None
IndexEntry(id=13229, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=62, line=21), position=None
IndexEntry(id=13230, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=79, line=12), position=None
IndexEntry(id=13231, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=80, line=9), position=None
IndexEntry(id=13232, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=80, line=10), position=None
IndexEntry(id=13233, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=80, line=19), position=None


In [10]:
word_dict["niños"].index_ref

[(24, 10), (30, 6), (30, 15), (62, 21), (79, 12), (80, 9), (80, 10), (80, 19)]

In [11]:
from sqlalchemy import select
stmt = select(Word).where(Word.word == "quedaran")

for word in session.scalars(stmt):
    print(word)

Word(id=2627, word='quedaran', comment=None)


In [12]:
stmt = select(IndexEntry).where(IndexEntry.word_id == 2627)

for entry in session.scalars(stmt):
    print(entry)

IndexEntry(id=18271, word=Word(id=2627, word='quedaran', comment=None), word_id=2627, lesson=80, line=9), position=None


In [13]:
from time import time
from lesson_parser import Word_properties
stmt = select(Word)
new_dict = {}
ti = time()
for word in session.scalars(stmt):
    index_ref = []
    for entry in word.index_ref_entries:
        index_ref.append((entry.lesson, entry.line))
    new_dict[word.word] = Word_properties(word.tonic_accent, index_ref)
tf = time()
print(f"time to fetch word dictionary from db : {tf-ti}")

time to fetch word dictionary from db : 2.3888509273529053


In [14]:
from lesson_parser import get_words
from tonic_accent import get_sentences

from sqlalchemy.orm import Session
engine.echo=False
with Session(engine) as session:
    for lesson in range(1, 101):
        sentences = get_sentences(lesson)
        for line, sentence in enumerate(sentences):
            s_obj = Sentence(
                lesson = lesson,
                line = line,
                sentence = sentence
            )
            session.add(s_obj)
    session.commit()


In [24]:
from sqlalchemy import and_
stmt = select(Sentence).where(and_(Sentence.line == 11, Sentence.lesson == 50))
ti = time()
for s_obj in session.scalars(stmt):
    print(s_obj.lesson, s_obj.line, s_obj.sentence)
tf = time()
print(f"time to fetch word dictionary from db : {tf-ti}")

50 11 S10-¡Ay, cuánto se lo agradezco! Es usted muy amable. 
time to fetch word dictionary from db : 0.0016322135925292969


In [37]:
word = "tampoco"
stmt = select(Word).where(Word.word == word)
word_obj = session.scalars(stmt).one()
print(word_obj)
for entry in word_obj.index_ref_entries:
    stmt = select(Sentence).where(and_(Sentence.lesson == entry.lesson, Sentence.line == entry.line))
    for s_obj in session.scalars(stmt):
        print(f"{s_obj.lesson}, {s_obj.line} : {s_obj.sentence}")
    

Word(id=263, word='tampoco', comment=None)
11, 5 : S04-Yo tampoco, mamá, estoy haciendo la tarea. 
11, 13 : T02-Yo tampoco te puedo ayudar estoy haciendo la tarea. 
13, 5 : S04-Lo siento, yo tampoco tengo ganas de ir de copas. 
13, 13 : T04-Mi mujer tampoco tiene ganas de ir de copas. 
22, 5 : S04-Yo tampoco me encuentro bien, hija mía. 
33, 13 : S12-Es que tampoco tenemos mucho dinero, ¿sabe? 
49, 5 : S04-Yo tampoco escribo mucho, sabes, tal vez una postal de vez en cuando
81, 5 : S04-Yo tampoco. Si se tratara de eso, yo también estaría en contra. 
84, 6 : S05-No, yo tampoco soy partidaria de cambiar las costumbres españolas. 
86, 12 : S11-¿Ese? Y de marido perfecto tampoco No para de hacerle ojitos a la del tercero A. 
96, 11 : S10-y por lo tanto tampoco una numerosa población indígena. 
